#I. Load the required libraries

In [2]:
import tensorflow as tf
import numpy as np
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score, KFold, train_test_split, learning_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from google.colab import drive
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report,multilabel_confusion_matrix,confusion_matrix,accuracy_score, mean_squared_error, explained_variance_score, make_scorer, mean_absolute_error, r2_score
from sklearn.neural_network import MLPRegressor
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.utils import np_utils
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.svm import SVR
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV 
from random import randint
from sklearn.model_selection import RandomizedSearchCV


##II. Dataset Preparation

In [5]:
drive.mount('/content/gdrive')
%cd '/content/gdrive/My Drive/'

dataframe = pandas.read_csv("ML_Project/faultdataset-Bus7.csv")
dataframe.sample(n=5)

Mounted at /content/gdrive
/content/gdrive/My Drive


,Time,Bus 1_Vm,Bus 2_Vm,Bus 3_Vm,Bus 4_Vm,Bus 5_Vm,Bus 6_Vm,Bus 7_Vm,Bus 8_Vm,Bus 9_Vm,Bus 1_PA,Bus 2_PA,Bus 3_PA,Bus 4_PA,Bus 5_PA,Bus 6_PA,Bus 7_PA,Bus 8_PA,Bus 9_PA,Freq 1,Freq 2,Freq 3,Fault_duration_location
9993,0.02,1.00,1.00,1.00,0.99,0.98,1.00,0.99,1.00,0.96,0.00,0.17,0.08,-0.04,-0.07,0.03,0.01,0.07,-0.08,0.00,0.00,0.00,3.05
1986,1.50,1.18,1.19,1.20,1.17,1.16,1.20,1.17,1.18,1.14,0.88,0.68,0.89,0.78,0.73,0.80,0.69,0.68,0.68,0.01,0.00,0.01,8.15
852,2.05,1.19,1.34,1.23,1.20,1.19,1.23,1.24,1.28,1.19,-0.57,-0.88,-0.52,-0.68,-0.72,-0.64,-0.80,-0.85,-0.81,0.00,-0.08,0.00,2.25
3830,1.88,1.15,1.11,1.11,1.12,1.10,1.12,1.10,1.11,1.08,0.35,0.60,0.53,0.33,0.32,0.47,0.44,0.49,0.31,-0.03,-0.05,-0.04,4.30
9730,3.00,1.03,1.00,1.00,1.01,0.99,1.01,0.99,1.00,0.98,0.44,0.61,0.51,0.39,0.36,0.46,0.44,0.50,0.36,-0.08,-0.07,-0.08,1.20


In [6]:
#convert the data into an array 
dataset = dataframe.values
X = dataset[0:,1:22].astype(float)
y = dataset[0:,22].astype(float)


#Split the data into training and testing parts 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# III. Training Models

##K-Nearest Neighbors (KNN) Model##

###Hypertuning: GridSearchCV

In [4]:
knn_param={
    'n_neighbors':[5,9,17,21,53,101],
    'weights':['uniform','distance'],
    'metric': ['eucludian','manhattan']
}

In [ ]:
knn=RandomizedSearchCV(KNeighborsRegressor(),knn_param,verbose=1,cv=5,n_jobs=-1, scoring='neg_mean_squared_error', return_train_score='true')
knn_results=knn.fit(X,y)
knn_results.best_score_
knn_results.best_estimator_
knn_results.best_params_

###Testing resutls

In [6]:
#KNN
knn = KNeighborsRegressor(metric='manhattan', n_neighbors=101, weights='distance').fit(X_train, y_train) 
knn_predictions = knn.predict(X_test)  
mae_knn=mean_absolute_error(y_test,knn_predictions)
mse_knn=mean_squared_error(y_test,knn_predictions)
r2_knn=r2_score(y_test,knn_predictions)
print("\nKNN model")
print("Mean Absolute Error (MAE) "+str(mae_knn)) 
print("Mean Squared Error (MSE) "+str(mse_knn)) 
print("R Squared "+str(r2_knn))



KNN model
Mean Absolute Error (MAE) 1.959538605363425
Mean Squared Error (MSE) 5.732849993232834
R Squared 0.1371234378434647


##Random Forest Regressor (RFR) Model##

In [7]:
rf_param = { 
    'n_estimators': [100, 1000, 10000],
    'max_features': ['None', 'sqrt', 'log2'],
    'max_depth' : [4,6]
}

In [ ]:
rf_gridsearch=GridSearchCV(RandomForestRegressor(), rf_param, verbose=1, cv=5, n_jobs=-1, scoring='neg_mean_squared_error', return_train_score='true')
rf_results=rf_gridsearch.fit(X,y)
rf_results.best_score_
rf_results.best_estimator_
rf_results.best_params_

###Testing resutls

In [11]:
#RF
rf = RandomForestRegressor(n_estimators=1000, max_features='auto' )
rf.fit(X_train,y_train)
rf_prediction=rf.predict(X_test)
mae_rf=mean_absolute_error(y_test,rf_prediction)
mse_rf=mean_squared_error(y_test,rf_prediction)
r2_rf=r2_score(y_test,rf_prediction)
print("\nRandom Forest Model: 1000 Estimators")
print("Mean Absolute Error (MAE) "+str(mae_rf)) 
print("Mean Squared Error (MSE) "+str(mse_rf)) 
print("R Squared "+str(r2_rf))

#learning curve
#train_sizes_rf, train_scores_rf, validation_scores_rf = learning_curve(rf,X,y, cv=5, scoring = 'neg_mean_squared_error')


Random Forest Model: 1000 Estimators
Mean Absolute Error (MAE) 1.2479003525723618
Mean Squared Error (MSE) 3.2123357373862746
R Squared 0.5164971661842903


##Naive Bayes (NB) Model

In [ ]:
nb_param={
    'n_iter':[100,200, 300, 400, 500],
    'lambda_1':[ 1,1e-06, 1e-02, 1e-04],
    'alpha_1': [ 1,1e-06, 1e-02, 1e-04]
}
nb=GridSearchCV(linear_model.BayesianRidge(),nb_param,verbose=1,cv=5,n_jobs=-1, scoring= 'neg_mean_squared_error',  return_train_score='true')
nb_results=nb.fit(X,y)
nb_results.best_score_
nb_results.best_estimator_
nb_results.best_params_

Fitting 5 folds for each of 80 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 122 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:   10.1s finished


{'alpha_1': 1, 'lambda_1': 1e-06, 'n_iter': 100}

###Testing resutls

In [ ]:
#NB
nb =linear_model.BayesianRidge(n_iter=300, lambda_1= 1e-06, alpha_1=1).fit(X_train, y_train) 
gnb_predictions = nb.predict(X_test) 
mae_nb=mean_absolute_error(y_test,gnb_predictions)
mse_nb=mean_squared_error(y_test,gnb_predictions)
r2_nb=r2_score(y_test,gnb_predictions)
print("\nNaive Bayesian model")
print("Mean Absolute Error (MAE) "+str(mae_nb)) 
print("Mean Squared Error (MSE) "+str(mse_nb)) 
print("R Squared "+str(r2_nb))


Naive Bayesian model
Mean Absolute Error (MAE) 2.066736544323986
Mean Squared Error (MSE) 6.082267773215985
R Squared 0.08453102515097632


##Decision Tree (DT) Model

In [ ]:
dt_param={
    'random_state':[0,1,2, 3, 4, 5],
    'min_samples_leaf':[ 1,6, 12, 18],
    'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson']
    
}
  
dt=GridSearchCV(DecisionTreeRegressor(), dt_param,verbose=1,cv=5,n_jobs=-1, scoring= 'neg_mean_squared_error', return_train_score='true')
dt_results=dt.fit(X,y)
dt_results.best_score_
dt_results.best_estimator_
dt_results.best_params_

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 243 tasks      | elapsed:   26.5s
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:   29.1s finished


{'criterion': 'friedman_mse', 'min_samples_leaf': 18, 'random_state': 3}

###Testing resutls

In [ ]:
#DT
dt = DecisionTreeRegressor(min_samples_leaf=12,  max_features=20, random_state= 4 ).fit(X_train, y_train) 
dtree_prediction = dt.predict(X_test) 
mae_dt=mean_absolute_error(y_test,dtree_prediction) 
mse_dt=mean_squared_error(y_test,dtree_prediction) 
r2_dt=r2_score(y_test,dtree_prediction) 

print("\nDecision tree model") 
print("Mean Absolute Error (MAE) "+str(mae_dt)) 
print("Mean Squared Error (MSE) "+str(mse_dt)) 
print("R Squared "+str(r2_dt))


Decision tree model
Mean Absolute Error (MAE) 1.4584692992304888
Mean Squared Error (MSE) 4.198934733969239
R Squared 0.36799979552161777


##Deep Neural Network (DNN) Model

###Testing resutls

In [ ]:
#DNN

def dnn_model():
    model = Sequential()
    model.add(Dense(1000, input_dim=21, kernel_initializer='normal', activation='relu')) #500 units has 8.2% loss
    model.add(Dense(500, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1000, kernel_initializer='normal', activation='relu'))
#     model.add(Dense(20, kernel_initializer='normal', activation='relu'))
#     model.add(Dense(20, kernel_initializer='normal', activation='relu'))
#     model.add(Dense(70, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1000, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

from keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier

dnn_model_regressor = KerasRegressor(build_fn=dnn_model)
history = dnn_model_regressor.fit(X_train, y_train)
dnn_model_regressor_predict = dnn_model_regressor.predict(X_test)
mae_dnn=mean_absolute_error(y_test, dnn_model_regressor_predict)
mse_dnn=mean_squared_error(y_test, dnn_model_regressor_predict)
r2_dnn=r2_score(y_test, dnn_model_regressor_predict)
print("\nDNN with 3 hidden layers with 1000, 500, and 1000 units each and Relu activation function")
print("Mean Absolute Error (MAE) "+str(mae_dnn)) 
print("Mean Squared Error (MSE) "+str(mse_dnn)) 
print("R Squared "+str(r2_dnn)) 

583/583 [==============================] - 16s 26ms/step - loss: 7.6922

DNN with 3 hidden layers with 1000, 500, and 1000 units each and Relu activation function
Mean Absolute Error (MAE) 2.1393978775219304
Mean Squared Error (MSE) 6.331901800492066
R Squared 0.04695750560876932


##Neural Network (NN) Model

###Testing resutls

In [ ]:
NN1 = MLPRegressor(max_iter=1500, activation='relu', solver='adam', learning_rate_init=0.001, hidden_layer_sizes=450).fit(X_train,y_train)
NN1_prediction = NN1.predict(X_test)
mae_NN1=mean_absolute_error(y_test,NN1_prediction)
mse_NN1=mean_squared_error(y_test,NN1_prediction)
r2_NN1=r2_score(y_test,NN1_prediction)
print("NN with Relu activation function")
print("Mean Absolute Error (MAE) "+str(mae_NN1)) 
print("Mean Squared Error (MSE) "+str(mse_NN1)) 
print("R Squared "+str(r2_NN1))

NN with Relu activation function
Mean Absolute Error (MAE) 1.6318024853800575
Mean Squared Error (MSE) 4.2896934766144135
R Squared 0.35433929647982165


##Support Vector Machine (SVM) Model

###Testing resutls

In [ ]:
SVM_model_2 = SVR(kernel = 'rbf', C = 10, gamma = 'scale').fit(X_train, y_train) 
SVM_predictions_2 = SVM_model_2.predict(X_test)  
mae_SVM_2=mean_absolute_error(y_test,SVM_predictions_2)
mse_SVM_2=mean_squared_error(y_test,SVM_predictions_2)
r2_SVM_2=r2_score(y_test,SVM_predictions_2)
print("\nSVR with a rbf kernel")
print("Mean Absolute Error (MAE) "+str(mae_SVM_2)) 
print("Mean Squared Error (MSE) "+str(mse_SVM_2)) 
print("R Squared "+str(r2_SVM_2)) 


SVR with a rbf kernel
Mean Absolute Error (MAE) 1.9523634057282546
Mean Squared Error (MSE) 5.963108784918884
R Squared 0.10246617054867968


##Hoeffdin tree (HT) Model

###Download scikit-multiflow package

In [ ]:
!pip install -U git+https://github.com/scikit-multiflow/scikit-multiflow

In [ ]:
from skmultiflow.data import FileStream
from skmultiflow.trees import HoeffdingTreeClassifier
from skmultiflow.evaluation import EvaluatePrequential
from skmultiflow.data import WaveformGenerator

In [ ]:
stream =  FileStream("ML_Project/faultdataset-Bus7.csv")
stream.prepare_for_use()

/usr/local/lib/python3.7/dist-packages/skmultiflow/data/base_stream.py:194: FutureWarning: 'prepare_for_use' has been deprecated in v0.5.0 and will be removed in v0.7.0.
New instances of the Stream class are now ready to use after instantiation.
  category=FutureWarning)


###Testing resutls

In [ ]:
ht=GridSearchCV(HoeffdingTreeClassifier(),ht_param,verbose=1,cv=5,n_jobs=-1, scoring='neg_mean_squared_error', return_train_score='true')
ht_results=ht.fit(X,y)
ht_results.best_score_
ht_results.best_estimator_
ht_results.best_params_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 23.1min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 24.9min finished


##IV. Simulate Missing Data

####Generate missing data by percentage 10%

In [ ]:
#Generate missing data by percentage 10%
import warnings

def ampute_mcar(X_complete, missing_rate = 0.1):
    # Mask completly at random some values
    
    M = np.random.binomial(1, missing_rate, size = X_complete.shape)
    X_obs = X_complete.copy()
    np.putmask(X_obs, M, np.nan)
    print('Percentage of newly generated missing values: {}'.\
      format(np.round(np.sum(np.isnan(X_obs))/X_obs.size,3)))
    
    # warning if a full row is missing
    for row in X_obs:
        if np.all(np.isnan(row)):
            warnings.warn('Some row(s) contains only nan values.')
            break

    # warning if a full col is missing
    for col in X_obs.T:
        if np.all(np.isnan(col)):
            warnings.warn('Some col(s) contains only nan values.')
            break
            
    return X_obs
  
X_train_missing_data = ampute_mcar(X_train)

#Convert NAN values into 0 
from numpy import *
where_are_NaNs = isnan(X_train_missing_data)
X_train_missing_data[where_are_NaNs] = 0

Percentage of newly generated missing values: 0.099


####Re-Training RFR and KNN model including missing data

#### RFR model results

In [ ]:
#Computing the training processing time
import time
start_time = time.time()
#RF
rf = RandomForestRegressor(n_estimators=1000)
rf.fit(X_train_missing_data,y_train)
end_time=time.time()
training_time=end_time-start_time
print("Training time for 18634 samples "+str(training_time)) 

#Computing the testing processing time
start_time = time.time()
rf_prediction=rf.predict(X_test)
end_time=time.time()
testing_time=end_time-start_time
print("Testing time for 7986 samples "+str(testing_time)) 


mae_rf=mean_absolute_error(y_test,rf_prediction)
mse_rf=mean_squared_error(y_test,rf_prediction)
r2_rf=r2_score(y_test,rf_prediction)
print("\nRandom Forest model")
print("Mean Absolute Error (MAE) "+str(mae_rf)) 
print("Mean Squared Error (MSE) "+str(mse_rf)) 
print("R Squared "+str(r2_rf))

Training time for 37461 samples 193.36716294288635
Testing time for 16055 samples 1.9241337776184082

Random Forest model
Mean Absolute Error (MAE) 1.3859734851339194
Mean Squared Error (MSE) 3.569380918747213
R Squared 0.46275671963659515


####KNN model results

In [ ]:
#Re-Training model with missing data

#Computing the training processing time
import time
start_time = time.time()
#KNN
knn = KNeighborsRegressor()
knn.fit(X_train_missing_data,y_train)
end_time=time.time()
training_time=end_time-start_time
print("Training time for 18634 samples "+str(training_time)) 

#Computing the testing processing time
start_time = time.time()
knn_prediction=knn.predict(X_test)
end_time=time.time()
testing_time=end_time-start_time
print("Testing time for 7986 samples "+str(testing_time)) 


mae_knn=mean_absolute_error(y_test,knn_prediction)
mse_knn=mean_squared_error(y_test,knn_prediction)
r2_knn=r2_score(y_test,knn_prediction)
print("\nKNN model")
print("Mean Absolute Error (MAE) "+str(mae_knn)) 
print("Mean Squared Error (MSE) "+str(mse_knn)) 
print("R Squared "+str(r2_knn))

Training time for 37461 samples 0.11109185218811035
Testing time for 16055 samples 1.4329698085784912

KNN model
Mean Absolute Error (MAE) 2.063934385174055
Mean Squared Error (MSE) 6.45684510393188
R Squared 0.028151737402050125
